# 1. Cài đặt thư viện cần thiết
  - code model bằng pytorch

In [ ]:
!pip install nltk
!pip install torch                                          
!pip install PyCU

     |████████████████████████████████| 305 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for PyCU: filename=pycu-0.2.0-py3-none-any.whl size=2486 sha256=8d49387d68272729db4df9040ec187a4b348945ab9097043c8b4a6cbe961bd24
  Stored in directory: /root/.cache/pip/wheels/95/48/8c/bb32a5fc8f6f7019b7349fd27998007ea7ed6fcf250c55e92e
  Created wheel for pyicu: filename=PyICU-2.9-cp37-cp37m-linux_x86_64.whl size=1375782 sha256=c113bac31c4112f078f81f6b63d1652c2a7f4eb452c0030117fba6a3e2f8a897
  Stored in directory: /root/.cache/pip/wheels/28/88/93/6c1b06361e4cbd4e7f793fb456729f69798f9aa3fc2a791cd7
Successfully built PyCU pyicu


# 2. Import các thư viện cần thiết


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re # thư viện regex để bắt regex, dùng trong xử lí các kí tự dư thừa
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 3. Đọc dữ liệu



In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP/data_en.csv", names=["text"])
data.dropna(inplace=True) # xóa nan trong dữ liệu
data.reset_index(inplace=True, drop =True) # reset lại index
data.head(5) #lấy 5 dòng đầu của dữ liệu ra nhìn

,text
0,Please put the dustpan in the broom closet
1,Be quiet for a moment.
2,Read this
3,Tom persuaded the store manager to give him ba...
4,Friendship consists of mutual understanding


In [ ]:
dta = data[:1000] # lấy 1000 dòng đầu tiên của dữ liệu
dta.head()

,text
0,Please put the dustpan in the broom closet
1,Be quiet for a moment.
2,Read this
3,Tom persuaded the store manager to give him ba...
4,Friendship consists of mutual understanding


# 4. Tiền xử lí dữ liệu

## 1. Bỏ các kí tự dư thừa như [],/?#@$%^,...

In [ ]:
def preprocessing_data(data):
  lst_sentences = []
  for sentence in data:
    temp = re.findall("[^\d\s]\w+", sentence) # bắt regex, [^\d\s]\w+: không lấy số, không lấy kí tự không phải chữ
    s = " ".join(re.findall("\w+[^\d\s]", " ".join(temp))) # vì temp là 1 list nên cần phải nối lại thành câu bằng 1 dấu cách nên xài join
    lst_sentences.append(s)
  return lst_sentences

In [ ]:
dta = preprocessing_data(dta.text)
dta[:3]

['Please put the dustpan in the broom closet',
 'Be quiet for moment',
 'Read this']

# 5. Tạo từ điển

In [ ]:
import icu
def make_dictionary(data):
  dic_val = {"jjjj" : -1}
  dic_inx = {-1 : "jjjj"}
  wordset = []
  collator = icu.Collator.createInstance(icu.Locale('de_DE.UTF-8'))
  for s in data:
    wordset.extend(word_tokenize(s))
  vals = sorted(set(wordset),key=collator.getSortKey)
  
  for val, key in  enumerate(vals):
    dic_val[key]=val+1
    dic_inx[val+1]=key
  return dic_val, dic_inx

# 6. Tách mỗi câu thành 1 list gồm các từ đã tokenize

In [ ]:
def token_sentence(data):
  df = []
  for sentence in data:
    df.append(word_tokenize(sentence))
  return df
df = token_sentence(dta)
df[:3]

[['Please', 'put', 'the', 'dustpan', 'in', 'the', 'broom', 'closet'],
 ['Be', 'quiet', 'for', 'moment'],
 ['Read', 'this']]

In [ ]:
# mã hóa toàn bộ các câu thành index và chuyển ngược lại các câu từ index thành câu
def senten_to_index(vocab, data):
  wids = []
  for sentence in data:
    wids.append([vocab[w] for w  in sentence])
  return wids
s_t_i = senten_to_index(vocab, df)
s_t_i[:3]

[[1036, 1083, 1369, 403, 675, 1369, 178, 261],
 [119, 1093, 523, 875],
 [1109, 1390]]

In [ ]:
senten_to_index(idx_vocab, s_t_i[:3])

[['Please', 'put', 'the', 'dustpan', 'in', 'the', 'broom', 'closet'],
 ['Be', 'quiet', 'for', 'moment'],
 ['Read', 'this']]

# 7. Padding cho các câu có độ dài bằng nhau

In [ ]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 4.3 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
max_len = max([len(i) for i in s_t_i])
data1 = tf.keras.preprocessing.sequence.pad_sequences(s_t_i, max_len, padding='post')
data1

array([[1036, 1083, 1369, ...,    0,    0,    0],
       [ 119, 1093,  523, ...,    0,    0,    0],
       [1109, 1390,    0, ...,    0,    0,    0],
       ...,
       [ 613, 1537,    5, ...,    0,    0,    0],
       [1412, 1512,  731, ...,    0,    0,    0],
       [1412, 1512,  720, ...,    0,    0,    0]], dtype=int32)

#7. Tạo input và target, ngram với n = 3

In [ ]:
n = 3
def generate_sample(data, n = 3):
  context_word = []
  target = []
  # for seq in data:
  for sentence in data:
    trigram = [((sentence[i], sentence[i + 1]), sentence[i + 2])
            for i in range(len(sentence) - 2)]
    context_word.extend([trigram[i][0] for i in range(len(trigram))])
    target.extend([trigram[i][-1] for i in range(len(trigram))])
  return context_word, target
  # for seq in data:
  

In [ ]:
context_word, target = generate_sample(data1, n=4)
context_word[0]

(1036, 1083)

# 8. Xây dựng model


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NGRAM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGRAM, self).__init__()
    '''
      context_size : size của context_word, ở đây mỗi context_word có size bằng 2
      vocab_size: size của dictionary mình tạo
      embedding_dim: số chiều của mỗi chữ(đã mã hóa thành số) được mã hóa thành vector, vd chữ "Please" được mã hóa sang index là 1036, 1036 sẽ được
      embedding chuyển thành 1 vector có chiều là embedding_dim
      linear1: lớp hidden layer
      linear2: output layer
      log_softmax: lớp softmax tính xác suất để dự đoán có target là gì
    '''
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size*embedding_dim, 128) #context_size * embedding_dim
    self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1)) #embedding input, đưa input về shape(1, -1)
    out1 = F.relu(self.linear1(embeds)) #đưa qua hàm kích hoạt, ở đây chọn hàm relu
    out2 = self.linear2(out1) #lớp ẩn 2
    log_probs = F.log_softmax(out2, dim = 1) 
    return log_probs

  def predict(self,input):
    context_idxs = torch.tensor(input, dtype=torch.long)
    res = self.forward(context_idxs)
    res_arg = torch.argmax(res)
    res_val, res_ind = res.sort(descending=True)
    res_val = res_val[0][:1]
    res_ind = res_ind[0][:1]
    return res_val, res_ind


In [ ]:
import torch.optim as optim
vocab_size = len(vocab)
EMBEDDING_DIM = 256
CONTEXT_SIZE = 2 # size cua 1 context
losses = []
loss_function = nn.NLLLoss()
model = NGRAM(vocab_size,  EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [ ]:

for epoch in range(200):
    total_loss = 0


    for context, tg in zip(context_word, target):
      #step1 : chuẩn bị dữ liệu, dữ liệu đầu vào cần là 1 tensor(1 dạng dữ liệu giống mảng của numy nhưng thuộc pytorch hỗ trợ)
        context_idxs = torch.tensor(context)
      #step2: đặt lại gradient descent thành 0
        model.zero_grad()
      #step3: chạy hàm forward
        log_probs = model(context_idxs)
      #step4: tính loss function
        loss = loss_function(log_probs, torch.tensor([tg], dtype=torch.long))
      #step5: thực hiện lan truyền ngược(mục đích để tối ưu hàm loss) và cập nhật lại các tham số của mô hình, optimize bằng gradient 
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 
    print(total_loss)
    losses.append(total_loss)


34844.45597091496
31138.25330015569
29257.50455398646
27809.535237354914
26639.062041326702
25622.36857660562
24701.26531364129
23838.56067553547
23006.39984451258
22187.607403896425
21369.35498182825
20553.146763439043
19736.88794639729
18925.158428201863
18113.860921604
17308.13622086567
16506.355581187992
15713.403290380336
14928.774938520144
14152.079988600279
13390.528943665971
12643.529185579477
11915.071494602409
11210.272660413455
10529.92312058663
9877.695250589093
9254.88872070501
8666.09215812655
8110.167344154264
7590.873821620851
7109.750461808293
6668.891471118228
6266.470716758837
5905.152523510559
5580.266156298518
5293.727191585893
5039.9948651948625
4818.443162003298
4624.35571528509
4455.8374545224815
4308.851386486239
4181.557239460034
4070.0963978325644
3972.9356400309243
3888.899634771202
3814.811620330932
3749.154839911338
3690.7472378096063
3636.7633442798424
3591.452366714683
3550.336065681738
3513.1037522016763
3478.6003001412755
3447.181732509447
3419.5744088

In [ ]:
res_val,res_ind = model.predict(context_word[2])
for arg in zip(res_val,res_ind):
    #print(arg)
    print([(key,val,arg[0]) for key,val in vocab.items() if val == arg[1]])

[('in', 675, tensor(-0.0183, grad_fn=<UnbindBackward0>))]


In [ ]:
context_word[2]

(1369, 403)

In [ ]:
dta[:5]

['Please put the dustpan in the broom closet',
 'Be quiet for moment',
 'Read this',
 'Tom persuaded the store manager to give him back his money',
 'Friendship consists of mutual understanding']

In [ ]:
def predict_text(text):
  text = text.replace("[X]","jjjj" ) # chuyển kí tự [X] thành jjjj là 1 từ không có nghĩa trong từ điển và jjjj có index  = -1
  lst = token_sentence([text]) # tách từ
  lst2idx = senten_to_index(vocab, lst) #chuyển chữ thành index
  tensor = tf.keras.preprocessing.sequence.pad_sequences(lst2idx, max_len, padding='post') #đưa về dạng tensor để có thể đưa và mô hình
  x, y = generate_sample(lst, n = 3) # tạo context_word và target
  df = pd.DataFrame({'X': x, 'target': y})                 
  X_test = df.loc[df.target == "jjjj"].X.values[-1] #lấy context_word có từ target là jjjj
  X_test = tuple(vocab[i] for i in X_test)
  res_val, res_ind = model.predict(X_test) #dự đoán
  predict_word = idx_vocab[res_ind.numpy()[0]]
  return text.replace("jjjj",predict_word) #trả về nguyên câu

In [ ]:
text_goc = dta[0]
text = 'Please put [X] dustpan in the broom closet'
print("Cau goc: ", text_goc)
print("Cau dung de test: ", text)
print("Cau sau khi dien tu du doan: ", predict_text(text))

Cau goc:  Please put the dustpan in the broom closet
Cau dung de test:  Please put [X] dustpan in the broom closet
Cau sau khi dien tu du doan:  Please put the dustpan in the broom closet


In [ ]:
import random

correct = 0
all_sentence = len(dta)
#test 20 câu
df_test = token_sentence(dta[:20])

for i in df_test:
  if len(i) < 3:
    all_sentence -= 1
    continue
  print('câu ban đầu = ',i)
  # ramdom một từ trong câu để loại ra sau đó dự đoán dựa trên
  # mô hình đã xây dựng
  randomIndex = random.randint(0, len(i) - 3)
  test = i
  tag = test[randomIndex+2]
  test[randomIndex+2] = '[X]'
  print('câu để test = ',test)
  sentence_test = ''
  for w in test:
    sentence_test += w + ' '
  sentence_test = sentence_test[:-1]
  try:
    kq = predict_text(sentence_test)
    li = list(kq.split(" "))
    print('câu dự đoán = ',li)

    if tag == li[randomIndex+2]:
      correct += 1
  except Exception as e:
    continue
  print()
correct

câu ban đầu =  ['Please', 'put', 'the', 'dustpan', 'in', 'the', 'broom', 'closet']
câu để test =  ['Please', 'put', 'the', 'dustpan', 'in', 'the', '[X]', 'closet']
câu dự đoán =  ['Please', 'put', 'the', 'dustpan', 'in', 'the', 'shade', 'closet']

câu ban đầu =  ['Be', 'quiet', 'for', 'moment']
câu để test =  ['Be', 'quiet', '[X]', 'moment']
câu dự đoán =  ['Be', 'quiet', 'for', 'moment']

câu ban đầu =  ['Tom', 'persuaded', 'the', 'store', 'manager', 'to', 'give', 'him', 'back', 'his', 'money']
câu để test =  ['Tom', 'persuaded', 'the', 'store', 'manager', 'to', '[X]', 'him', 'back', 'his', 'money']
câu dự đoán =  ['Tom', 'persuaded', 'the', 'store', 'manager', 'to', 'give', 'him', 'back', 'his', 'money']

câu ban đầu =  ['Friendship', 'consists', 'of', 'mutual', 'understanding']
câu để test =  ['Friendship', 'consists', '[X]', 'mutual', 'understanding']
câu dự đoán =  ['Friendship', 'consists', 'of', 'mutual', 'understanding']

câu ban đầu =  ['Are', 'you', 'going', 'to', 'come', 'to

11